In [1]:
import os
from dotenv import load_dotenv
from functools import lru_cache
from crewai import LLM
from datetime import datetime
# Load environment variables from .env file
load_dotenv()

# Set the environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI1_API_KEY = os.getenv("GEMINI1_API_KEY")

OPENROUTER_API_KEY2=os.getenv("OPENROUTER_API_KEY2")
OPENAI_API_BASE=os.getenv("OPENAI_API_BASE")

os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

print("API Keys loaded successfully.")

@lru_cache(maxsize=1)
def initialize_llm():
    return LLM(
        model="openrouter/z-ai/glm-4.5-air:free",
        api_key=OPENROUTER_API_KEY2,
        base_url=os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1"),
        temperature=0.4,        # lower randomness for agentic use            # enable streaming if helpful
    )

@lru_cache(maxsize=1)
def initialize_llm1():
    """Initialize and cache the LLM instance to avoid repeated initializations."""
    return LLM(
        model="gemini/gemini-2.0-flash",
        provider="google",
        api_key=GEMINI_API_KEY
    )

import requests
import json
from crewai.tools import tool          # decorator
from crewai_tools import SerperDevTool # web-search tool

# Initialize the web search tool
search_tool = SerperDevTool()

# Tool 1: Human Input Tool
# This tool pauses the execution and asks for human input.
@tool("Human Input Tool")
def human_input_tool(question: str) -> str:
    """Asks a human for input. The agent should use this to ask for a budget if one is not provided."""
    return input(f"\n{question}\n")

def geocode_city(city: str) -> tuple[float, float] | None:
    url = "https://geocoding-api.open-meteo.com/v1/search"
    resp = requests.get(url, params={"name": city, "count": 1, "language": "en"})
    resp.raise_for_status()
    results = resp.json().get("results")
    if results:
        return results[0]["latitude"], results[0]["longitude"]
    return None

# Tool 2: Weather Tool (Updated for Forecast)
bad_weather_codes = [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]
desc_map = {
    0: "clear sky", 1: "mainly clear", 2: "partly cloudy", 3: "overcast", 45: "foggy", 51: "light drizzle", 
    61: "rain", 71: "snow", 95: "thunderstorm"
}

@tool("Weather Tool")
def open_meteo_weather_tool(city: str, start_date: str, end_date: str) -> str:
    """Returns weather forecast for a city between start_date and end_date using Open-Meteo."""
    coords = geocode_city(city)
    if not coords:
        return f"Sorry, I couldn’t find coordinates for {city}."
    lat, lon = coords
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,weathercode",
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "auto"
    }
    try:
        r = requests.get(url, params=params, timeout=8)
        r.raise_for_status()
        data = r.json()
        daily = data["daily"]
        forecast_lines = [f"Weather forecast for {city.title()} from {start_date} to {end_date}:"]
        bad_weather_dates = []
        for i in range(len(daily["time"])):
            date = daily["time"][i]
            max_temp = daily["temperature_2m_max"][i]
            min_temp = daily["temperature_2m_min"][i]
            code = daily["weathercode"][i]
            desc = desc_map.get(code, "unknown")
            forecast_lines.append(f"- {date}: {min_temp}°C to {max_temp}°C, {desc}")
            if code in bad_weather_codes:
                bad_weather_dates.append(date)
        if bad_weather_dates:
            forecast_lines.append("\nNote: Bad weather (rain, snow, or thunderstorms) expected on: " + ", ".join(bad_weather_dates))
        return "\n".join(forecast_lines)
    except Exception as e:
        return f"Error fetching Open-Meteo data: {e}"

# Tool 3: Currency Conversion Tool
def get_conversion_rate(from_currency: str, to_currency: str) -> float | None:
    """Helper function to get a numerical conversion rate."""
    try:
        url = f"https://open.er-api.com/v6/latest/{from_currency}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data['rates'][to_currency]
    except Exception:
        return None

# Your existing tool can now be simplified
@tool("Currency Conversion Tool")
def currency_conversion_tool(from_currency: str, to_currency: str) -> str:
    """
    Returns the numerical conversion rate from one currency to another as a string.
    """
    rate = get_conversion_rate(from_currency, to_currency)
    if rate:
        return str(rate)
    return f"Error converting currency. Ensure currency codes are correct."


print("Tools created successfully.")

# Define country to currency mapping
country_to_currency = {
    'Australia': 'AUD',
    'Brazil': 'BRL',
    'Canada': 'CAD',
    'China': 'CNY',
    'France': 'EUR',
    'Germany': 'EUR',
    'India': 'INR',
    'Italy': 'EUR',
    'Japan': 'JPY',
    'Mexico': 'MXN',
    'Singapore': 'SGD',
    'South Africa': 'ZAR',
    'Spain': 'EUR',
    'Sri Lanka': 'LKR',
    'Switzerland': 'CHF',
    'Thailand': 'THB',
    'United Arab Emirates': 'AED',
    'United Kingdom': 'GBP',
    'United States': 'USD',
    # Add more as needed
}

def calculate_nights(dates: str) -> int:
    """Calculates the number of nights for a given date range."""
    try:
        start_str, end_str = dates.split(' to ')
        start_date = datetime.strptime(start_str.strip(), '%Y-%m-%d')
        end_date = datetime.strptime(end_str.strip(), '%Y-%m-%d')
        # The number of nights is the difference in days
        num_nights = (end_date - start_date).days
        return max(0, num_nights)  # Return 0 if dates are invalid or same day
    except (ValueError, IndexError):
        return 0

def invoke_agent(location, interests, budget, num_people, travel_dates, preferred_currency):
    """Invokes the travel agent with the given inputs."""

    try:
        budget_amount_str, budget_currency = budget.strip().split()
        budget_amount = float(budget_amount_str)
        budget_currency = budget_currency.upper()
    except ValueError:
        print("Error: Invalid budget format. Please use 'AMOUNT CURRENCY' (e.g., '250 USD').")
        return

    budget_in_usd = budget_amount
    if budget_currency != 'USD':
        print(f"CONVERTING BUDGET: Original budget is {budget_amount} {budget_currency}.")
        rate_to_usd = get_conversion_rate(budget_currency, 'USD')
        if rate_to_usd:
            budget_in_usd = budget_amount * rate_to_usd
            print(f"--> Normalized budget for internal use is approximately {budget_in_usd:.2f} USD.")
        else:
            print(f"Warning: Could not convert {budget_currency} to USD. Calculations may be inaccurate.")

    # Determine local currency
    country = location.split(',')[-1].strip()
    local_currency = country_to_currency.get(country, 'USD')
    target_currency = preferred_currency if preferred_currency else local_currency

    # Determine if accommodation is needed
    # Calculate the number of nights
    num_nights = calculate_nights(travel_dates)

    # Determine if accommodation is needed and create the instruction
    accommodation_instruction = ""
    if num_nights > 0:
        accommodation_instruction = f"""
        **Crucially, you MUST research and suggest one suitable accommodation for a {num_nights}-night stay.**
        Given the interest in 'villa', prioritize finding a villa for {num_people} people.
        The total cost of the accommodation for the entire {num_nights}-night stay must be factored into the total budget of {budget}.
        """

    # Initialize LLM
    llm_model = initialize_llm()
    llm1_model = initialize_llm1()

    from crewai import Agent

    # Agent 1: Local Data Agent
    local_data_agent = Agent(
        role="Local Data Specialist",
        goal="Fetch weather and currency data for the travel destination.",
        backstory="An analyst providing real-time travel insights.",
        tools=[open_meteo_weather_tool, currency_conversion_tool],
        llm=llm1_model,
        verbose=True
    )

    # Agent 2: Web Search Agent (City Expert)
    city_expert_agent = Agent(
        role='Expert City Researcher',
        goal='Efficiently find a specific number of activities and accommodation within a budget.',
        backstory='A travel enthusiast who finds the best spots tailored to your needs, focusing on speed and accuracy.',
        tools=[search_tool],
        llm=llm_model,
        verbose=True,
        max_iter=10,  # Hard limit on the number of execution loops (thinking -> tool -> observation)
        allow_delegation=False
    )

    # Agent 3: Budget Verifier Agent
    budget_verifier_agent = Agent(
        role='Budget Verification Analyst',
        goal='Critically analyze the researched activities and their estimated costs against the user-provided budget. Provide a clear "go" or "no-go" verdict with justification.',
        backstory='A meticulous financial analyst with a knack for sniffing out hidden costs and ensuring travel plans are financially sound. You are firm but fair.',
        tools=[],
        llm=llm1_model,
        allow_delegation=False,
        verbose=True
    )

    # Agent 4: Travel Concierge Agent
    travel_concierge_agent = Agent(
        role='Head Travel Concierge',
        goal='Synthesize all gathered information into a cohesive, beautifully formatted travel itinerary with weather insights and converted costs.',
        backstory='A world-class concierge from a five-star hotel, known for creating personalized and delightful travel experiences.',
        tools=[currency_conversion_tool],  # Added for cost conversion
        llm=llm1_model,
        allow_delegation=False,
        verbose=True
    )

    print("Agents defined successfully.")

    from crewai import Task

    # Task 1: Get local data (weather forecast and currency conversion)
    task_get_local_data = Task(
        description=f"""Fetch the weather forecast and the USD to local currency conversion rate for the trip to {location}.

        The travel dates are: {travel_dates}.

        You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.
        Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the weather tool.
        """,
        expected_output="A summary of the weather forecast for the specified dates and the USD to local currency conversion rate.",
        agent=local_data_agent
    )

    # Task 2: Find city information
    task_find_city_info = Task(
        description=f"""
        For a group of {num_people} traveling to {location}, create a focused travel plan based on a total budget of **{budget_in_usd:.2f} USD** and interests in '{interests}'.

        **IMPORTANT CONTEXT USAGE:** You will receive context from a data specialist that includes a real-time currency conversion rate. If you find prices online in a local currency (e.g., INR, LKR), you **must use the precise conversion rate provided in your context** to convert them to USD for your analysis and final JSON output. This is more accurate than using your general knowledge.

        The TOTAL estimated cost of all researched items (in USD) must not exceed this budget and also should be close to this budget.

        **Your instructions are to be highly efficient. Aim to use the web search tool no more than 2-3 times.**

        Your research output MUST contain the following specific items:
        1.  **Exactly 2-3** top attractions or restaurants or entertainment places that match the interests.
        2.  {accommodation_instruction} 

        Your final answer MUST be a single JSON string. This JSON object should contain a key "items" which is a list of dictionaries, and a key "total_estimated_cost_usd".
        Each dictionary in the "items" list must have the keys: "type" (string, e.g., "accommodation" or "activity"), "name" (string), "description" (string), and "cost_usd" (number).
        """,
        expected_output="""A single, valid JSON string that can be directly parsed. Example format: 
        '{"items": [{"type": "accommodation", "name": "Mirissa Beach Villa", "description": "A beautiful villa with a pool for 4 guests.", "cost_usd": 150}, {"type": "activity", "name": "Whale Watching Tour", "description": "A 4-hour whale watching excursion.", "cost_usd": 80}], "total_estimated_cost_usd": 230}'
        """,
        agent=city_expert_agent,
        context=[task_get_local_data]
    )

    # Task 3: Verify the budget
    task_verify_budget = Task(
        description=f"Analyze the research from the city expert for a group of {num_people}. The total available budget is **{budget_in_usd:.2f} USD**. Sum up the total estimated cost of ALL items (activities and accommodation) provided by the researcher.Compare this total to the available USD budget. Provide a clear 'go' or 'no-go' verdict with a brief justification. The user's original budget was '{budget}'.",
        expected_output="A budget feasibility verdict (Go/No-Go) comparing the total estimated cost in USD against the total available budget in USD.",
        agent=budget_verifier_agent,
        context=[task_find_city_info]
    )

    # Task 4: Compile the final report
    task_compile_report = Task(
        description=f"""
        Create a final, human-readable travel itinerary for {num_people} people for a trip to {location} from {travel_dates}.

        You will receive structured data in a JSON string format from the city expert's context. Your first step is to parse this JSON to access the list of activities and accommodation.

        Your report must:
        1.  First, use the Currency Conversion Tool ONCE to get the numerical conversion rate from USD to {target_currency}.
        2.  Iterate through the items from the parsed JSON. For each item, use the rate to convert its 'cost_usd' to {target_currency}.
            **When displaying the cost, show ONLY the final converted amount. Do NOT show the original USD cost or the mathematical calculation used to arrive at the final price.**
            For example, instead of writing "Cost: 100 USD x 301.95 = 30,195 LKR", you MUST write "Cost: 30,195 LKR".
        3.  Synthesize the parsed items into a cohesive, daily plan.
        4.  **Important:** Do NOT display the 'USD to {target_currency}' conversion rate in the report if the user's original budget was already provided in {target_currency}. Only show the conversion rate if the original budget currency was different from the final report currency.
        5.  Incorporate the budget verification verdict from the context.
        6.  Include the weather insights. If bad weather is predicted, suggest alternative indoor activities.
        7.  Format the entire output as a beautiful and exciting markdown report. Display all final costs ONLY in {target_currency}.
        """,
        expected_output=f"A complete, beautifully formatted markdown report with a travel plan, budget analysis, and weather insights. All costs must be in {target_currency} and must not show any calculations.",
        agent=travel_concierge_agent,
        context=[task_verify_budget, task_get_local_data, task_find_city_info]
    )

    print("Tasks created successfully.")

    from crewai import Crew, Process
    from IPython.display import Markdown

    # Create the Crew
    travel_crew = Crew(
        agents=[local_data_agent, city_expert_agent, budget_verifier_agent, travel_concierge_agent],
        tasks=[task_get_local_data, task_find_city_info, task_verify_budget, task_compile_report],
        process=Process.sequential,
        verbose=True
    )

    # Kick off the crew's work!
    result = travel_crew.kickoff()

    # Print the final result
    if hasattr(result, 'raw') and isinstance(result.raw, str):
        display(Markdown(result.raw))
    else:
        print(f"Error: Expected a CrewOutput object with a 'raw' attribute containing a string, but got {type(result)}")

invoke_agent("Mirissa, Sri Lanka", "entertainment, beach and affordable villa with pool. we need lunch dinner breakfast to eat in an affordable way", "32000 LKR", 4, "2025-08-05 to 2025-08-06", "")


API Keys loaded successfully.


/Users/nithilathawalampitiya/Documents/Projects/TravelAgent/myenv-crew/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


Tools created successfully.
CONVERTING BUDGET: Original budget is 32000.0 LKR.
--> Normalized budget for internal use is approximately 105.95 USD.
Agents defined successfully.
Tasks created successfully.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1488a0f9-b8a2-41a8-a7ee-844ffcc7b408                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Data Specialist                                                                                   │
│                                                                                                                 │
│  Task: Fetch the weather forecast and the USD to local currency conversion rate for the trip to Mirissa, Sri    │
│  Lanka.                                                                                                         │
│                                                                                                                 │
│          The travel dates are: 2025-08-05 to 2025-08-06.                                                        │
│                                                                                                                 │
│          You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.       │
│          Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the  │
│  weather tool.                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  301.947388                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Head Travel Concierge                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Mirissa, Sri Lanka: 2-Day Itinerary (August 5-6, 2025)                                                       │
│                                                                                                                 │
│  ## Budget Verdict: No-Go                                                                                       │
│                                                                                                                 │
│  The estimated cost exceeds the provided budget.                                                                │
│                                                                                                                 │
│  ## Weather Overview:                                                                                           │
│                                                                                                                 │
│  *   **August 5, 2025:** Overcast, with temperatures ranging from 26.9°C to 29.6°C.                             │
│  *   **August 6, 2025:** Unknown weather conditions, with temperatures ranging from 27.0°C to 29.4°C. Bad       │
│  weather is expected.                                                                                           │
│                                                                                                                 │
│  ## Day 1: August 5, 2025                                                                                       │
│                                                                                                                 │
│  *   **Accommodation:** Mirissa Beach Villa                                                                     │
│      *   Description: An affordable beachfront villa with a private pool, perfect for 4 guests. Located less    │
│  than 1 km from Weligama Beach with river and garden views.                                                     │
│      *   Cost: 22,646 LKR                                                                                       │
│  *   **Activity:** Mirissa Beach Relaxation                                                                     │
│      *   Description: Enjoy the beautiful Mirissa Beach, swim in the crystal-clear waters, and relax on the     │
│  sandy shore.                                                                                                   │
│      *   Cost: 0 LKR                                                                                            │
│  *   **Restaurant:** Local Breakfast                                                                            │
│      *   Description: Traditional Sri Lankan breakfast at a local eatery including rice and curry, fresh        │
│  fruits, and local tea.                                                                                         │
│      *   Cost: 1,208 LKR                                                                                        │
│  *   **Activity:** Parrot Rock Exploration                                                                      │
│      *   Description: Visit the iconic Parrot Rock in Mirissa for stunning views and photo opportunities.       │
│      *   Cost: 604 LKR                                 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 079e2092-ee87-4d10-88b2-4e12292180fb                                                                     │
│  Agent: Head Travel Concierge                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1488a0f9-b8a2-41a8-a7ee-844ffcc7b408                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Mirissa, Sri Lanka: 2-Day Itinerary (August 5-6, 2025)                                                       │
│                                                                                                                 │
│  ## Budget Verdict: No-Go                                                                                       │
│                                                                                                                 │
│  The estimated cost exceeds the provided budget.                                                                │
│                                                                                                                 │
│  ## Weather Overview:                                                                                           │
│                                                                                                                 │
│  *   **August 5, 2025:** Overcast, with temperatures ranging from 26.9°C to 29.6°C.                             │
│  *   **August 6, 2025:** Unknown weather conditions, with temperatures ranging from 27.0°C to 29.4°C. Bad       │
│  weather is expected.                                                                                           │
│                                                                                                                 │
│  ## Day 1: August 5, 2025                                                                                       │
│                                                                                                                 │
│  *   **Accommodation:** Mirissa Beach Villa                                                                     │
│      *   Description: An affordable beachfront villa with a private pool, perfect for 4 guests. Located less    │
│  than 1 km from Weligama Beach with river and garden views.                                                     │
│      *   Cost: 22,646 LKR                                                                                       │
│  *   **Activity:** Mirissa Beach Relaxation                                                                     │
│      *   Description: Enjoy the beautiful Mirissa Beach, swim in the crystal-clear waters, and relax on the     │
│  sandy shore.                                                                                                   │
│      *   Cost: 0 LKR                                                                                            │
│  *   **Restaurant:** Local Breakfast                                                                            │
│      *   Description: Traditional Sri Lankan breakfast at a local eatery including rice and curry, fresh        │
│  fruits, and local tea.                                                                                         │
│      *   Cost: 1,208 LKR                                                                                        │
│  *   **Activity:** Parrot Rock Exploration                                                                      │
│      *   Description: Visit the iconic Parrot Rock in 

```markdown
# Mirissa, Sri Lanka: 2-Day Itinerary (August 5-6, 2025)

## Budget Verdict: No-Go

The estimated cost exceeds the provided budget.

## Weather Overview:

*   **August 5, 2025:** Overcast, with temperatures ranging from 26.9°C to 29.6°C.
*   **August 6, 2025:** Unknown weather conditions, with temperatures ranging from 27.0°C to 29.4°C. Bad weather is expected.

## Day 1: August 5, 2025

*   **Accommodation:** Mirissa Beach Villa
    *   Description: An affordable beachfront villa with a private pool, perfect for 4 guests. Located less than 1 km from Weligama Beach with river and garden views.
    *   Cost: 22,646 LKR
*   **Activity:** Mirissa Beach Relaxation
    *   Description: Enjoy the beautiful Mirissa Beach, swim in the crystal-clear waters, and relax on the sandy shore.
    *   Cost: 0 LKR
*   **Restaurant:** Local Breakfast
    *   Description: Traditional Sri Lankan breakfast at a local eatery including rice and curry, fresh fruits, and local tea.
    *   Cost: 1,208 LKR
*   **Activity:** Parrot Rock Exploration
    *   Description: Visit the iconic Parrot Rock in Mirissa for stunning views and photo opportunities.
    *   Cost: 604 LKR
*   **Restaurant:** Lunch at Beachside Cafe
    *   Description: Affordable beachside lunch with fresh seafood and local specialties.
    *   Cost: 1,812 LKR
*   **Restaurant:** Dinner at Local Restaurant
    *   Description: Authentic Sri Lankan dinner at a budget-friendly local restaurant.
    *   Cost: 1,510 LKR

## Day 2: August 6, 2025

*   **Activity:** Whale Watching Tour
    *   Description: A 3-5 hour whale watching excursion from Mirissa Harbor, offering the chance to see Blue Whales and Dolphins. Includes wildlife tax.
    *   Cost: 6,491 LKR

**Due to the expected bad weather, the following indoor activities are suggested as alternatives:**

*   **Cooking Class:** Learn to prepare traditional Sri Lankan dishes.
*   **Spa Treatment:** Enjoy a relaxing Ayurvedic massage.
*   **Visit a local gem museum:** Sri Lanka is famous for its sapphires and other gemstones.

```

In [2]:
# # --- Define User Inputs ---
# location = 'Mirissa, Sri Lanka'
# interests = 'entertainment, beach and affordable villa with pool. we need lunch dinner breakfast to eat in an affordable way'
# budget = '250 USD'
# num_people = 4 
# travel_dates = '2025-08-05 to 2025-08-06'  # Static travel dates
# preferred_currency = ''  # Leave empty for local currency, or set to 'GBP', etc.

In [3]:
# from functools import lru_cache
# from crewai import LLM
# import os
# from datetime import datetime
# from crewai_tools import SerperDevTool # web-search tool


# OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# OPENAI_API_BASE=os.getenv("OPENAI_API_BASE")

# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# print("API Keys loaded successfully.")

# @lru_cache(maxsize=1)
# def initialize_llm():
#     return LLM(
#         model="openrouter/z-ai/glm-4.5-air:free",
#         api_key=os.getenv("OPENROUTER_API_KEY"),
#         base_url=os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1"),
#         temperature=0.4,        # lower randomness for agentic use            # enable streaming if helpful
#     )

# # Define country to currency mapping
# country_to_currency = {
#     'Sri Lanka': 'LKR',
#     'United States': 'USD',
#     'United Kingdom': 'GBP',
#     # Add more as needed
# }
# # Initialize the web search tool
# search_tool = SerperDevTool()

# location = 'Mirissa, Sri Lanka'
# interests = 'entertainment, beach and affordable villa with pool'
# budget = '250 USD'
# num_people = 4 
# travel_dates = '2025-08-05 to 2025-08-06'  # Static travel dates
# preferred_currency = ''  # Leave empty for local currency, or set to 'GBP', etc.

# # Determine local currency
# country = location.split(',')[-1].strip()
# local_currency = country_to_currency.get(country, 'USD')
# target_currency = preferred_currency if preferred_currency else local_currency

# def calculate_nights(dates: str) -> int:
#     """Calculates the number of nights for a given date range."""
#     try:
#         start_str, end_str = dates.split(' to ')
#         start_date = datetime.strptime(start_str.strip(), '%Y-%m-%d')
#         end_date = datetime.strptime(end_str.strip(), '%Y-%m-%d')
#         # The number of nights is the difference in days
#         num_nights = (end_date - start_date).days
#         return max(0, num_nights)  # Return 0 if dates are invalid or same day
#     except (ValueError, IndexError):
#         return 0

# # Determine if accommodation is needed
# # Calculate the number of nights
# num_nights = calculate_nights(travel_dates)

# # Determine if accommodation is needed and create the instruction
# accommodation_instruction = ""
# if num_nights > 0:
#     accommodation_instruction = f"""
#     **Crucially, you MUST research and suggest one suitable accommodation for a {num_nights}-night stay.**
#     Given the interest in 'villa', prioritize finding a villa for {num_people} people.
#     The total cost of the accommodation for the entire {num_nights}-night stay must be factored into the total budget of {budget}.
#     """

# # Initialize LLM
# llm_model = initialize_llm()

# from crewai import Agent

# # Agent 3: Web Search Agent (City Expert)
# city_expert_agent = Agent(
#     role='Expert City Researcher',
#     goal='Efficiently find a specific number of activities and accommodation within a budget.',
#     backstory='A travel enthusiast who finds the best spots tailored to your needs, focusing on speed and accuracy.',
#     tools=[search_tool],
#     llm=llm_model,
#     verbose=True,
#     max_iter=5,  # Hard limit on the number of execution loops (thinking -> tool -> observation)
#     allow_delegation=False
# )

# from crewai import Task

# # Task 3: Find city information
# task_find_city_info = Task(
#     description=f"""
#     For a group of {num_people} traveling to {location}, create a focused travel plan based on a total budget of {budget} and interests in '{interests}'.

#     **Your instructions are to be highly efficient. Aim to use the web search tool no more than 2-3 times.**

#     Your research output MUST contain the following specific items:
#     1.  **Exactly 2-3** top attractions or restaurants or entertainment places that match the interests.
#     2.  {accommodation_instruction} # This is the dynamic instruction from the previous step

#     Once you have gathered information for these items, consolidate them into a single list with descriptions and estimated USD costs for the group.
#     The TOTAL estimated cost of all researched items must not exceed the budget.
#     Do not search for more items than requested. Your final answer should be the complete, consolidated list.
#     """,
#     expected_output="A concise list of 2-3 attractions/restaurants/entertainment places and accommodation (if required), with descriptions and a total estimated cost in USD. The agent should stop once these items are found.",
#     agent=city_expert_agent
# )

# from crewai import Crew, Process
# from IPython.display import Markdown

# # Create the Crew
# travel_crew = Crew(
#     agents=[city_expert_agent],
#     tasks=[task_find_city_info],
#     process=Process.sequential,
#     verbose=True
# )

# # Kick off the crew's work!
# result = travel_crew.kickoff()